### Example1

In [1]:
import pymloc
import numpy as np
import jax.numpy as jnp
import warnings
warnings.filterwarnings('ignore')

#### Creating the object

First, we need to create a parameter dependent optimal control problem.

We need `variables`, `objective` and `constraint`.

#### Creating the variables object
The variables for the different levels are defined as follows.

In [2]:
from pymloc.model.variables import InputStateVariables
from pymloc.model.variables import NullVariables
from pymloc.model.variables import ParameterContainer
from pymloc.model.variables.time_function import Time
from pymloc.model.domains import RNDomain


loc_vars = InputStateVariables(1, 1, time=Time(0., 2.))
hl_vars = ParameterContainer(1, domain=RNDomain(1))
variables2 = (hl_vars, loc_vars)

ll_vars = NullVariables()

#### Creating the control system
The parameter dependent control system is defined by

In [3]:
from pymloc.model.control_system.parameter_dae import LinearParameterControlSystem

def e(p, t):
    return jnp.array([[1.]])


def a(p, t):
    return jnp.array([[-1.]])


def b(p, t):
    return jnp.array([[1.]])


def c(p, t):
    return jnp.array([[1.]])


def d(p, t):
    return jnp.array([[0.]])


def f(p, t):
    return jnp.array([0.])


param_control = LinearParameterControlSystem(ll_vars, *variables2, e, a, b, c,
                                             d, f)

#### Creating the constraint object

In [4]:
from pymloc.model.optimization.parameter_optimal_control import ParameterLQRConstraint


def initial_value(p):
    return jnp.array([2.])



time = Time(0., 2.)

pdoc_constraint = ParameterLQRConstraint(*variables2, param_control,
                                         initial_value)


#### Creating the objective function

The objective function is defined by

In [5]:
from pymloc.model.optimization.parameter_optimal_control import ParameterLQRObjective

def q(p, t):
    return jnp.array([[p**2. - 1.]])


def s(p, t):
    return jnp.zeros((1, 1))


def r(p, t):
    return jnp.array([[1.]])


def m(p):
    return jnp.array([[0.]])


time = Time(0., 2.)
pdoc_objective = ParameterLQRObjective(*variables2, time, q, s, r, m)


#### Create the parameter dependent optimal control object

In [6]:
from pymloc.model.optimization.parameter_optimal_control import ParameterDependentOptimalControl

pdoc_object = ParameterDependentOptimalControl(*variables2, pdoc_objective,
                                               pdoc_constraint)


The neccessary conditions can be obtained as follows

In [7]:
neccessary_conditions = pdoc_object.get_bvp()
e = neccessary_conditions.dynamical_system.e(2., 3.)
a = neccessary_conditions.dynamical_system.a(2., 3.)
print("E =\n {},\nA =\n {}".format(e, a))

E =
 [[ 0.  1.  0.]
 [-1.  0.  0.]
 [-0.  0.  0.]],
A =
 [[ 0. -1.  1.]
 [-1.  3.  0.]
 [ 1.  0.  1.]]


#### Obtaining sensitivity values



##### Reference solution
A reference solution is given analytically and defined by



In [8]:
def refsol(theta, t0, tf, t, x01):
    refsol = np.array(
        [[(1 / ((-1 + theta + np.exp(2 * (-t0 + tf) * theta) *
                 (1 + theta))**2)) * np.exp(-(t + t0) * theta) * x01 *
          (np.exp(-2 * (t0 - 2 * tf) * theta) * (1 + theta)**2 *
           (1 + t + t0 *
            (-1 + theta) - t * theta) - np.exp(2 * (t - t0 + tf) * theta) *
           (1 + theta)**2 *
           (1 + 2 * tf + t * (-1 + theta) + t0 *
            (-1 + theta) - 2 * tf * theta) - np.exp(2 * t * theta) * 2 *
           (-1 + theta)**2 * (1 + t * (1 + theta) - t0 *
                              (1 + theta)) + np.exp(2 * tf * theta) *
           (-1 + theta)**2 * (1 - t * (1 + theta) - t0 * (1 + theta) + 2 * tf *
                              (1 + theta)))],
         [(1 / ((-1 + theta + np.exp(2 * (-t0 + tf) * theta) *
                 (1 + theta))**2)) * np.exp(-(t + t0) * theta) * x01 *
          (np.exp(2 * t * theta) * (t - t0) *
           (-1 + theta)**2 + np.exp(-2 * t0 * theta + 4 * tf * theta) *
           (-t + t0) * (1 + theta)**2 + np.exp(2 * tf * theta) *
           (-2 + t + t0 - 2 * tf -
            (t + t0 - 2 * tf) * theta**2) + np.exp(2 * (t - t0 + tf) * theta) *
           (2 - t - t0 + 2 * tf + (t + t0 - 2 * tf) * theta**2))]])
    return refsol


##### Computed solution

Run the default sensitivities solver (adjoint computation) for different tolerances and collect data in `results` and `iresults`.

In [9]:
import logging
logger = logging.getLogger()
logging.getLogger().setLevel(logging.INFO)
logger.handlers[0].filters[0].__class__.max_level = 3

results = np.zeros((8, 2))
iresults = np.zeros((8, 2), dtype=np.int)

for tol_exp in range(8):
    sens = pdoc_object.get_sensitivities()
    tol = 10**(-tol_exp)
    logger.info("Initialized with tol = {}".format(tol))
    sens.init_solver(abs_tol=tol, rel_tol=tol)
    sol = sens.solve(parameters=np.array(2.), tau=1.)
    rsol = refsol(2., 0., 2., 1., 2.)
    ref = np.block([[rsol], [-rsol[0]]])
    atol = np.linalg.norm(ref - sol(1.))
    rtol = np.linalg.norm((ref - sol(1.))) / np.linalg.norm(ref)
    evals = len(sol.params["time_grid"]) - 3
    logger.info(
        "Solution tolerances:\nrtol: {}\natol: {}\nadditional_evaluations: {}"
        .format(rtol, atol, evals))
    results[tol_exp] = [rtol, atol]
    iresults[tol_exp] = [-tol_exp, evals]
 
    assert np.allclose(ref, sol(1.), rtol=rtol, atol=atol)


Initialized with tol = 1
	Starting solver AdjointSensitivitiesSolver
	Current option values:
	abs_tol: 1
	rel_tol: 1
	max_iter: 10
	Compute sensitivity at tau = 1.0
		Starting solver MultipleShooting
		Current option values:
		abs_tol: 0.16666666666666666
		rel_tol: 0.16666666666666666
		max_iter: 10
		MultipleShooting solver initialized with
		
		        shooting_nodes: [0.  0.5 1.  1.5 2. ]
		
		        boundary_nodes: (0.0, 2.0)
		
		Computing solution in the interval (0.0, 0.5)
		Computing solution in the interval (0.5, 1.0)
		Computing solution in the interval (1.0, 1.5)
		Computing solution in the interval (1.5, 2.0)
		Computing inhomogeneous solution in the interval (0.0, 0.5)
		Computing inhomogeneous solution in the interval (0.5, 1.0)
		Computing inhomogeneous solution in the interval (1.0, 1.5)
		Computing inhomogeneous solution in the interval (1.5, 2.0)
		Computing inhomogeneous solution in the interval (0.0, 0.5)
		Computing inhomogeneous solution in the interval (0.5, 1.

		Computing solution in the interval (0.0, 0.5)
		Computing solution in the interval (0.5, 1.0)
		Computing solution in the interval (1.0, 1.5)
		Computing solution in the interval (1.5, 2.0)
		Computing inhomogeneous solution in the interval (0.0, 0.5)
		Computing inhomogeneous solution in the interval (0.5, 1.0)
		Computing inhomogeneous solution in the interval (1.0, 1.5)
		Computing inhomogeneous solution in the interval (1.5, 2.0)
		Computing inhomogeneous solution in the interval (0.0, 0.5)
		Computing inhomogeneous solution in the interval (0.5, 1.0)
		Computing inhomogeneous solution in the interval (1.0, 1.5)
		Computing inhomogeneous solution in the interval (1.5, 2.0)
	Assembling adjoint sensitivity boundary value problem...
	Solving adjoint boundary value problem...
		Starting solver MultipleShooting
		Current option values:
		abs_tol: 0.00016666666666666666
		rel_tol: 0.00016666666666666666
		max_iter: 10
		MultipleShooting solver initialized with
		
		        shooting_nod

		Computing solution in the interval (1.5, 2.0)
		Computing inhomogeneous solution in the interval (0.0, 0.5)
		Computing inhomogeneous solution in the interval (0.5, 1.0)
		Computing inhomogeneous solution in the interval (1.0, 1.5)
		Computing inhomogeneous solution in the interval (1.5, 2.0)
		Computing inhomogeneous solution in the interval (0.0, 0.5)
		Computing inhomogeneous solution in the interval (0.5, 1.0)
		Computing inhomogeneous solution in the interval (1.0, 1.5)
		Computing inhomogeneous solution in the interval (1.5, 2.0)
	Assembling adjoint sensitivity boundary value problem...
	Solving adjoint boundary value problem...
		Starting solver MultipleShooting
		Current option values:
		abs_tol: 1.6666666666666665e-07
		rel_tol: 1.6666666666666665e-07
		max_iter: 10
		MultipleShooting solver initialized with
		
		        shooting_nodes: [0. 1. 2.]
		
		        boundary_nodes: (0.0, 1.0, 2.0)
		
		Computing solution in the interval (0.0, 1.0)
		Computing solution in the inter

Format the results for usage in latex

In [10]:
from tabulate import tabulate
print(tabulate(results, tablefmt='latex')+ "\n")
print(tabulate(iresults, tablefmt='latex'))

\begin{tabular}{rr}
\hline
 0.553269    & 0.152016    \\
 0.634564    & 0.174352    \\
 0.0228938   & 0.00629029  \\
 0.00387713  & 0.00106528  \\
 0.000936385 & 0.000257281 \\
 0.000150897 & 4.14603e-05 \\
 2.31734e-05 & 6.36711e-06 \\
 4.49058e-06 & 1.23383e-06 \\
\hline
\end{tabular}

\begin{tabular}{rr}
\hline
  0 &  35 \\
 -1 &  35 \\
 -2 &  54 \\
 -3 &  54 \\
 -4 & 119 \\
 -5 & 170 \\
 -6 & 158 \\
 -7 & 228 \\
\hline
\end{tabular}
